In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dense, Flatten, MaxPooling2D
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.cluster import KMeans
import numpy as np

In [2]:
train_dir = 'Dataset/Train'
test_dir = 'Dataset/test'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_gen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    vertical_flip = True
)

In [ ]:
train_data = data_gen.flow_from_directory(
    train_dir,
    target_size = (224, 224)
)

test_data = data_gen.flow_from_directory(
    test_dir,
    target_size = (224, 224)
)

In [ ]:
# Build a simple CNN model2
model2 = Sequential()

model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))

model2.add(Flatten())

model2.add(Dense(256, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(128, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(64, activation='relu'))

model2.add(Dense(9, activation='softmax'))  # Assuming 9 classes for classification

# Compile the model2
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model2 summary
model2.summary()


In [ ]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(train_data, epochs=10)

In [ ]:
model2.save('model.h5')

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
from tensorflow.keras.models import load_model

model2 = load_model('/kaggle/input/featureextractor/model.h5')

In [ ]:
model2.summary()

In [ ]:
feature_extractor_model = Model(inputs = model2.input, outputs = model2.get_layer('dense').output)

In [ ]:
train_path = '/kaggle/input/dermnet-and-skin-disease-9-classes/Merged Dataset/test'


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import InceptionResNetV2

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
data_gen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    vertical_flip = True
)

In [5]:
train_data = data_gen.flow_from_directory(
    train_dir,
    target_size = (224, 224)
)

test_data = data_gen.flow_from_directory(
    test_dir,
    target_size = (224, 224)
)

Found 26642 images belonging to 9 classes.
Found 2389 images belonging to 9 classes.


In [6]:
vgg_base_model = VGG19(include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 3s 0us/step


In [7]:
inception_resnet = InceptionResNetV2(
    include_top = False,
    input_shape = (224, 224, 3)
)

219055592/219055592 [==============================] - 9s 0us/step


In [8]:
for layer in vgg_base_model.layers:
    layer.trainable = False

In [9]:
for layer in inception_resnet.layers:
    layer.trainable = False

In [10]:
model1 = Sequential([
    vgg_base_model,
    GlobalAveragePooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(64, activation='relu'),
    Dense(9, activation='softmax')
])

In [11]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [2]:
model1.fit(train_data, validation_data=test_data, epochs=10)


Epoch 1/10
833/833 [==============================] - 242s 284ms/step - loss: 1.4151 - accuracy: 0.7103 - val_loss: 107.2386 - val_accuracy: 0.6001
Epoch 2/10
833/833 [==============================] - 149s 179ms/step - loss: 1.2428 - accuracy: 0.7717 - val_loss: 190.3661 - val_accuracy: 0.6223
Epoch 3/10
833/833 [==============================] - 148s 177ms/step - loss: 0.8600 - accuracy: 0.7908 - val_loss: 64.0182 - val_accuracy: 0.6495
Epoch 4/10
833/833 [==============================] - 148s 178ms/step - loss: 0.8202 - accuracy: 0.8855 - val_loss: 41.3609 - val_accuracy: 0.6621
Epoch 5/10
833/833 [==============================] - 153s 183ms/step - loss: 0.1391 - accuracy: 0.9093 - val_loss: 24.7617 - val_accuracy: 0.7363
Epoch 6/10
833/833 [==============================] - 148s 178ms/step - loss: 0.0999 - accuracy: 0.9163 - val_loss: 26.1198 - val_accuracy: 0.6629
Epoch 7/10
833/833 [==============================] - 146s 175ms/step - loss: 0.0259 - accuracy: 0.9368 - val_loss:

In [15]:
model1.save('vgg.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
model2 = Sequential([
    inception_resnet,
    GlobalAveragePooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(64, activation='relu'),
    Dense(9, activation='softmax')
])

In [17]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1]:
model2.fit(train_data, validation_data=test_data, epochs=10)


Epoch 1/10
833/833 [==============================] - 175s 194ms/step - loss: 1.3701 - accuracy: 0.7039 - val_loss: 1.6445 - val_accuracy: 0.6219
Epoch 2/10
833/833 [==============================] - 155s 186ms/step - loss: 1.1993 - accuracy: 0.7656 - val_loss: 1.5802 - val_accuracy: 0.6341
Epoch 3/10
833/833 [==============================] - 153s 184ms/step - loss: 1.1187 - accuracy: 0.7953 - val_loss: 1.5548 - val_accuracy: 0.6588
Epoch 4/10
833/833 [==============================] - 158s 190ms/step - loss: 1.0702 - accuracy: 0.8167 - val_loss: 1.4732 - val_accuracy: 0.6990
Epoch 5/10
833/833 [==============================] - 158s 190ms/step - loss: 1.0300 - accuracy: 0.8323 - val_loss: 1.4177 - val_accuracy: 0.7040
Epoch 6/10
833/833 [==============================] - 155s 186ms/step - loss: 0.9855 - accuracy: 0.8503 - val_loss: 1.3590 - val_accuracy: 0.7257
Epoch 7/10
833/833 [==============================] - 159s 190ms/step - loss: 0.9480 - accuracy: 0.8894 - val_loss: 1.3472 

In [19]:
model2.save('inception.h5')